# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 28 2022 11:37AM,249608,10,Enova-10897,Emerginnova,Released
1,Oct 28 2022 11:37AM,249608,10,Enova-10898,Emerginnova,Released
2,Oct 28 2022 11:36AM,249607,10,0086127221,ISDIN Corporation,Released
3,Oct 28 2022 11:36AM,249607,10,0086127225,ISDIN Corporation,Released
4,Oct 28 2022 11:36AM,249607,10,0086127236,ISDIN Corporation,Released
5,Oct 28 2022 11:36AM,249607,10,0086127244,ISDIN Corporation,Released
6,Oct 28 2022 11:36AM,249607,10,0086127295,ISDIN Corporation,Released
7,Oct 28 2022 11:36AM,249607,10,0086127296,ISDIN Corporation,Released
8,Oct 28 2022 11:14AM,249606,19,ADV80007158,"AdvaGen Pharma, Ltd",Released
9,Oct 28 2022 11:11AM,249605,19,N17826,"NAPP Technologies, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,249604,Released,1
28,249605,Released,1
29,249606,Released,1
30,249607,Released,6
31,249608,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249604,NaN,1.0
249605,NaN,1.0
249606,NaN,1.0
249607,NaN,6.0
249608,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249519,0.0,1.0
249520,0.0,1.0
249521,0.0,1.0
249523,0.0,1.0
249524,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249519,0,1
249520,0,1
249521,0,1
249523,0,1
249524,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249519,0,1
1,249520,0,1
2,249521,0,1
3,249523,0,1
4,249524,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249519,,1
1,249520,,1
2,249521,,1
3,249523,,1
4,249524,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 28 2022 11:37AM,249608,10,Emerginnova
2,Oct 28 2022 11:36AM,249607,10,ISDIN Corporation
8,Oct 28 2022 11:14AM,249606,19,"AdvaGen Pharma, Ltd"
9,Oct 28 2022 11:11AM,249605,19,"NAPP Technologies, LLC"
10,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc."
11,Oct 28 2022 10:50AM,249603,19,"Emersa Waterbox, LLC"
12,Oct 28 2022 10:44AM,249602,10,"Citieffe, Inc."
13,Oct 28 2022 10:38AM,249601,19,ACG North America LLC
15,Oct 28 2022 10:36AM,249595,10,Eywa Pharma Inc.
22,Oct 28 2022 10:34AM,249600,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 28 2022 11:37AM,249608,10,Emerginnova,,2
1,Oct 28 2022 11:36AM,249607,10,ISDIN Corporation,,6
2,Oct 28 2022 11:14AM,249606,19,"AdvaGen Pharma, Ltd",,1
3,Oct 28 2022 11:11AM,249605,19,"NAPP Technologies, LLC",,1
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",,1
5,Oct 28 2022 10:50AM,249603,19,"Emersa Waterbox, LLC",,1
6,Oct 28 2022 10:44AM,249602,10,"Citieffe, Inc.",,1
7,Oct 28 2022 10:38AM,249601,19,ACG North America LLC,1,1
8,Oct 28 2022 10:36AM,249595,10,Eywa Pharma Inc.,6,1
9,Oct 28 2022 10:34AM,249600,10,ISDIN Corporation,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 28 2022 11:37AM,249608,10,Emerginnova,2,
1,Oct 28 2022 11:36AM,249607,10,ISDIN Corporation,6,
2,Oct 28 2022 11:14AM,249606,19,"AdvaGen Pharma, Ltd",1,
3,Oct 28 2022 11:11AM,249605,19,"NAPP Technologies, LLC",1,
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",1,
5,Oct 28 2022 10:50AM,249603,19,"Emersa Waterbox, LLC",1,
6,Oct 28 2022 10:44AM,249602,10,"Citieffe, Inc.",1,
7,Oct 28 2022 10:38AM,249601,19,ACG North America LLC,1,1
8,Oct 28 2022 10:36AM,249595,10,Eywa Pharma Inc.,1,6
9,Oct 28 2022 10:34AM,249600,10,ISDIN Corporation,,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 28 2022 11:37AM,249608,10,Emerginnova,2,
1,Oct 28 2022 11:36AM,249607,10,ISDIN Corporation,6,
2,Oct 28 2022 11:14AM,249606,19,"AdvaGen Pharma, Ltd",1,
3,Oct 28 2022 11:11AM,249605,19,"NAPP Technologies, LLC",1,
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 28 2022 11:37AM,249608,10,Emerginnova,2.0,NaN
1,Oct 28 2022 11:36AM,249607,10,ISDIN Corporation,6.0,NaN
2,Oct 28 2022 11:14AM,249606,19,"AdvaGen Pharma, Ltd",1.0,NaN
3,Oct 28 2022 11:11AM,249605,19,"NAPP Technologies, LLC",1.0,NaN
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 28 2022 11:37AM,249608,10,Emerginnova,2.0,0.0
1,Oct 28 2022 11:36AM,249607,10,ISDIN Corporation,6.0,0.0
2,Oct 28 2022 11:14AM,249606,19,"AdvaGen Pharma, Ltd",1.0,0.0
3,Oct 28 2022 11:11AM,249605,19,"NAPP Technologies, LLC",1.0,0.0
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1497610,11.0,7.0
12,249594,1.0,0.0
15,1247831,76.0,31.0
16,1247717,4.0,5.0
19,1248019,5.0,1.0
21,748573,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1497610,11.0,7.0
1,12,249594,1.0,0.0
2,15,1247831,76.0,31.0
3,16,1247717,4.0,5.0
4,19,1248019,5.0,1.0
5,21,748573,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,11.0,7.0
1,12,1.0,0.0
2,15,76.0,31.0
3,16,4.0,5.0
4,19,5.0,1.0
5,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,11.0
1,12,Released,1.0
2,15,Released,76.0
3,16,Released,4.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Executing,7.0,0.0,31.0,5.0,1.0,0.0
Released,11.0,1.0,76.0,4.0,5.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Executing,7.0,0.0,31.0,5.0,1.0,0.0
1,Released,11.0,1.0,76.0,4.0,5.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Executing,7.0,0.0,31.0,5.0,1.0,0.0
1,Released,11.0,1.0,76.0,4.0,5.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()